In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
import bert
from transformers import TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from ipywidgets import IntProgress

print(tf.keras.__version__ ,'\t' , tf.__version__, "\t", np.__version__)

2.3.0-tf 	 2.2.0 	 1.18.5


In [4]:
import os
import re

# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df
train, test = download_and_load_datasets()

In [18]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

In [22]:
tokenized

0        [101, 1000, 4330, 2338, 1998, 13541, 1000, 200...
1        [101, 2009, 2001, 2200, 2540, 1011, 12959, 101...
2        [101, 2919, 1012, 2919, 1012, 2919, 1012, 2216...
3        [101, 2076, 1996, 2034, 1017, 3692, 7199, 5976...
4        [101, 2402, 3376, 9345, 1006, 2002, 5149, 1975...
                               ...                        
24995    [101, 3087, 2040, 2973, 2083, 1996, 5535, 1997...
24996    [101, 1996, 2034, 2112, 1010, 18178, 1999, 739...
24997    [101, 2023, 2003, 1037, 4416, 1999, 5048, 3185...
24998    [101, 1000, 24659, 1000, 2003, 1037, 17463, 14...
24999    [101, 2023, 2003, 2763, 2026, 5440, 3185, 1997...
Name: sentence, Length: 25000, dtype: object

In [24]:
import time
def create_tf_example(features,label):
    """
    Create tf example using features and label

    Args:
        features: list, feature list with format  ['idx','sentence']
        label: string, 

    Return:
        A binary-string of tf example.
        All proto messages can be serialized to a binary-string using the .SerializeToString method.
    """
    tf_example = tf.train.Example(features = tf.train.Features(feature = {
        'idx': tf.train.Feature(int64_list=tf.train.Int64List(value=[features[0]])),
        'sentence': tf.train.Feature(bytes_list=tf.train.BytesList(value=[features[1].encode('utf-8')])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }))
    return tf_example.SerializeToString()

def convert_csv_to_tfrecord(csv, file_name):
    """
    Convert the numpy arryes to tfrecord and write files

    Args:
        csv: numpy arrays, each row feed (features+label)
        file_name: location TFRecord to be saved 
    """
    start_time = time.time()
    writer = TFRecordWriter(file_name)
    for idx,row in enumerate(csv):
        # check the row retionality, raise error when missing value
        try:
            if row is None:
                raise Exception('Row Missing')
            if row[0] is None or row[1] is None or row[2] is None:
                raise Exception('Value Missing')
            if row[1].strip() is '':
                raise Exception('Utterance is empty')
            
            features, label = row[:-1],row[-1]
            example =  create_tf_example(features,label)
            writer.write(example)

        except Exception as inst:
            print(type(inst))
            print(inst.args)
            print(inst)
    writer.close()
    print(f"{file_name}: --- {(time.time() - start_time)} seconds ---")


<>:38: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:38: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-24-707dd93ba5d7>:38: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if row[1].strip() is '':


In [27]:
from tensorflow.python.lib.io.tf_record import TFRecordWriter
convert_csv_to_tfrecord(train, "./data/movie_train.tfrecord")
convert_csv_to_tfrecord(test, "./data/movie_test.tfrecord")


NotFoundError: ./data/movie_train.tfrecord; No such file or directory

In [5]:
from transformers import TFBertForSequenceClassification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import BertTokenizer, glue_convert_examples_to_features
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = glue_convert_examples_to_features(train, tokenizer, max_length=128, task='mrpc')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)

AttributeError: 'str' object has no attribute 'label'

In [9]:
glue_convert_examples_to_features?

Signature:
glue_convert_examples_to_features(
    examples: Union[List[transformers.data.processors.utils.InputExample], ForwardRef('tf.data.Dataset')],
    tokenizer: transformers.tokenization_utils.PreTrainedTokenizer,
    max_length: Union[int, NoneType] = None,
    task=None,
    label_list=None,
    output_mode=None,
)
Docstring:
Loads a data file into a list of ``InputFeatures``

Args:
    examples: List of ``InputExamples`` or ``tf.data.Dataset`` containing the examples.
    tokenizer: Instance of a tokenizer that will tokenize the examples
    max_length: Maximum example length. Defaults to the tokenizer's max_len
    task: GLUE task
    label_list: List of labels. Can be obtained from the processor using the ``processor.get_labels()`` method
    output_mode: String indicating the output mode. Either ``regression`` or ``classification``

Returns:
    If the ``examples`` input is a ``tf.data.Dataset``, will return a ``tf.data.Dataset``
    containing the task-specific features. 